In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1)
import matplotlib as mpl
import pandas as pd

In [2]:
state = 'colorado'
state_abbrev = 'CO'

In [3]:
#To estimate emissions for Colorado HPMS vehicle types, use emission factors (by road type) data sent by Dale Wells @ CDPHE.
emissionfactors = pd.read_excel('Transportation/EPA_MOVES2014a_StateEmissionFactors.xlsx')

In [9]:
#If time, use state-specific age distributions instead of MOVES default (have for CO and NM, waiting on NV).
agedist = pd.read_csv('Transportation/EPAMOVES2014_DefaultAgeDist_AnalysisYear2019.csv')

In [10]:
agedist

,sourceTypeID,yearID,ageID,ageFraction
0,Motorcycle,2019,0,0.070679
1,Motorcycle,2019,1,0.070490
2,Motorcycle,2019,2,0.069938
3,Motorcycle,2019,3,0.064916
4,Motorcycle,2019,4,0.059053
...,...,...,...,...
398,Combination Long-haul Truck,2019,26,0.004824
399,Combination Long-haul Truck,2019,27,0.002854
400,Combination Long-haul Truck,2019,28,0.002169
401,Combination Long-haul Truck,2019,29,0.001672


In [4]:
agedist['model_year'] = agedist['yearID']-agedist['ageID']
agedist.drop(columns=['yearID', 'ageID'], inplace=True)
agedist.set_index(['sourceTypeID', 'model_year'], inplace=True)

In [5]:
state_emissionfactors = emissionfactors[emissionfactors['State']== state.upper()]
state_emissionfactors.insert(2, 'weighted_emissions (g/mi)', 0)

In [6]:
for ind in state_emissionfactors.index:
    grams_per_mile_years = state_emissionfactors.loc[ind][4:]
    vehicle_type = state_emissionfactors.loc[ind]['Vehicle type']
    for sourceType in agedist.index.levels[0]:
        if str.lower(sourceType) in str.lower(vehicle_type):
            weighted_avg = np.dot(grams_per_mile_years[::-1], agedist.loc[sourceType])[0]
            state_emissionfactors.loc[ind, 'weighted_emissions (g/mi)'] = weighted_avg
state_emissionfactors.drop(state_emissionfactors.iloc[:, 4:], inplace = True, axis=1)
state_emissionfactors.rename(columns={'Pollutant (g/mi)': 'Pollutant'},inplace=True)
state_emissionfactors['Fuel Type'] = state_emissionfactors['Vehicle type'].str.split().str[-1]

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [11]:
state_emissionfactors

,State,Vehicle type,weighted_emissions (g/mi),Pollutant,Fuel Type
5,COLORADO,"Combination long-haul truck, diesel",3.404296,CO,diesel
56,COLORADO,"Combination short-haul truck, diesel",3.493977,CO,diesel
107,COLORADO,"Light commercial truck, diesel",2.969602,CO,diesel
158,COLORADO,"Light commercial truck, gasoline",5.711166,CO,gasoline
209,COLORADO,"Passenger car, diesel",3.810544,CO,diesel
...,...,...,...,...,...
6686,COLORADO,"Single unit long-haul truck, gasoline",0.609646,VOC (Evap),gasoline
6737,COLORADO,"Single unit short-haul truck, diesel",0.023129,VOC (Evap),diesel
6788,COLORADO,"Single unit short-haul truck, gasoline",0.930539,VOC (Evap),gasoline
6839,COLORADO,"Transit bus, CNG",NaN,VOC (Evap),CNG


In [7]:
state_emissionfactors['Vehicle type'].unique()

array(['Combination long-haul truck, diesel',
       'Combination short-haul truck, diesel',
       'Light commercial truck, diesel',
       'Light commercial truck, gasoline', 'Passenger car, diesel',
       'Passenger car, gasoline', 'Passenger truck, diesel',
       'Passenger truck, gasoline', 'Refuse truck, diesel',
       'School bus, diesel', 'School bus, gasoline',
       'Single unit long-haul truck, diesel',
       'Single unit long-haul truck, gasoline',
       'Single unit short-haul truck, diesel',
       'Single unit short-haul truck, gasoline', 'Transit bus, CNG',
       'Transit bus, diesel'], dtype=object)

In [8]:
pollutants = state_emissionfactors['Pollutant'].unique()

In [9]:
#Map vehicle types to reference fuel type.

mapping = {'Combination Long-Haul Truck': 'diesel' ,
           'Combination Short-Haul Truck': 'diesel',
           'Single Unit Long-Haul Truck': 'diesel',
           'Single Unit Short-Haul Truck': 'diesel',
           'Light Commercial Truck': 'gasoline', 
           'Passenger Car': 'gasoline',
           'Passenger Truck': 'gasoline',
           'Refuse Truck': 'diesel',
           'School Bus': 'diesel',
           'Transit Bus': 'diesel',
}

In [10]:
afv_multipliers = pd.read_csv('Transportation/AFV_ev_multipliers.csv')

In [11]:
afv_multipliers['afv_vehicle'] = afv_multipliers['Fuel']+ '_' + afv_multipliers['On-Road Application']

In [12]:
cols_to_drop = afv_multipliers.columns[5:-1]
afv_multipliers.drop(columns=cols_to_drop,inplace=True)

In [13]:
#Currently assumes PHEV and EREV multipliers are the same as HEVs. Andrew Burnham @ Argonne (GREET) confirmed this is a valid substitution.
#Currently assumes HEV and HHV multipliers are relative to default fuel type in mapping (above). Need to find emission factors for gasoline-HEV and gasoline-HHV heavy-duty vehicles.

afv_multipliers['emissions (g/mile)'] = 0
for afv in afv_multipliers['On-Road Application'].unique():
    afv_inds = afv_multipliers['On-Road Application']== afv
    comparison_vehicle_inds = state_emissionfactors['Vehicle type'].str.lower().str.contains(afv.lower())
    
    for pol in pollutants:
        pol_inds = afv_multipliers['Pollutant'] == pol
        afv_pol_inds = afv_inds & pol_inds
    
        for new_fuel in afv_multipliers['Fuel'].unique():
            new_fuel_inds = afv_multipliers['Fuel']==new_fuel
            afv_fuel_pol_inds = new_fuel_inds & afv_pol_inds

            if new_fuel in ['B20', 'B100']:
                comparison_fuel = 'diesel' 
            else:
                comparison_fuel = mapping[afv]
        
        
#         for torque in afv_multipliers[afv_fuel_inds]['Torque (lb-ft)'].unique():
#             afv_fuel_torque_inds = afv_multipliers[afv_fuel_inds]['Torque (lb-ft)'] == torque
#             if torque in ['gasoline', 'Diesel']:
#                 comparison_fuel = torque.lower()
        
            comparison_pol_inds = state_emissionfactors['Pollutant'] == pol
            comparison_fuel_inds = state_emissionfactors['Fuel Type'] == comparison_fuel
            comparison_ef_inds = comparison_vehicle_inds & comparison_pol_inds & comparison_fuel_inds

            comparison_ef = state_emissionfactors.loc[comparison_ef_inds,'weighted_emissions (g/mi)'].item()
            multiplier = afv_multipliers[afv_fuel_pol_inds]['Multiplier']
            afv_ef = comparison_ef * multiplier
            afv_multipliers.loc[afv_fuel_pol_inds,'emissions (g/mile)'] = afv_ef

In [14]:
state_emissionfactors.rename(columns={'weighted_emissions (g/mi)': 'emissions (g/mile)'}, inplace=True)
state_emissionfactors['Vehicle type'] = state_emissionfactors['Vehicle type'].str.lower()
state_emissionfactors['Vehicle type'] = state_emissionfactors['Vehicle type'].str.split(',').str[0]
state_emissionfactors['Reference/AFV'] = 'Reference'

<ipython-input-14-3dea39483fa9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_emissionfactors['Vehicle type'] = state_emissionfactors['Vehicle type'].str.lower()
<ipython-input-14-3dea39483fa9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_emissionfactors['Vehicle type'] = state_emissionfactors['Vehicle type'].str.split(',').str[0]
<ipython-input-14-3dea39483fa9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [15]:
afv_multipliers.rename(columns={'Fuel': 'Fuel Type','On-Road Application': 'Vehicle type'}, inplace=True)
afv_multipliers.drop(columns=['Torque (lb-ft)','Multiplier','afv_vehicle'],inplace=True)
afv_multipliers['State']=state.upper()
afv_multipliers['Vehicle type'] = afv_multipliers['Vehicle type'].str.lower()

In [16]:
afv_multipliers['Reference/AFV'] = 'AFV'

In [17]:
afv_multipliers

,Fuel Type,Vehicle type,Pollutant,emissions (g/mile),State,Reference/AFV
0,CNG,combination long-haul truck,CO,62.889234,NEW MEXICO,AFV
1,CNG,combination long-haul truck,NOx,1.342089,NEW MEXICO,AFV
2,CNG,combination long-haul truck,PM10,0.439067,NEW MEXICO,AFV
3,CNG,combination long-haul truck,PM10 (TBW),0.124242,NEW MEXICO,AFV
4,CNG,combination long-haul truck,PM2.5,0.403927,NEW MEXICO,AFV
...,...,...,...,...,...,...
1355,EREV,transit bus,PM10 (TBW),0.104000,NEW MEXICO,AFV
1356,EREV,transit bus,PM2.5,0.176728,NEW MEXICO,AFV
1357,EREV,transit bus,PM2.5 (TBW),0.013000,NEW MEXICO,AFV
1358,EREV,transit bus,VOC,0.526042,NEW MEXICO,AFV


In [18]:
diesel_inds = afv_multipliers['Fuel Type'] == 'Diesel'
gasoline_inds = afv_multipliers['Fuel Type'] == 'Gasoline'
afv_multipliers= afv_multipliers[~(diesel_inds & gasoline_inds)]

In [19]:
transit_inds = afv_multipliers['Vehicle type'] == 'transit bus'
fuel_inds = afv_multipliers['Fuel Type'] == 'CNG'
afv_multipliers= afv_multipliers[~(transit_inds & fuel_inds)]

In [20]:
all_efs = pd.concat([state_emissionfactors, afv_multipliers],ignore_index=True)

In [21]:
#Assumes school bus emission factors can be applied to school and intercity buses.
mapping = {
        'light commercial truck':'light duty trucks', 
        'refuse truck':'refuse trucks',
        'passenger car':'light duty autos', 
        'passenger truck':'light duty trucks',
        'transit bus' :'transit buses',
        'combination short-haul truck':'heavy duty trucks',
        'combination long-haul truck':'heavy duty trucks',
        'single unit long-haul truck' :'medium duty trucks',
        'single unit short-haul truck' :'medium duty trucks',
        'school bus' :'school and intercity buses'
        }

In [22]:
for vehicle in all_efs['Vehicle type'].unique():
    vehicle_inds = all_efs['Vehicle type']==vehicle
    evolved_category = mapping[vehicle]
    all_efs.loc[vehicle_inds, 'evolved_category'] = evolved_category

### Average emission factors across each vehicle subsector

In [23]:
all_efs

,State,Vehicle type,emissions (g/mile),Pollutant,Fuel Type,Reference/AFV,evolved_category
0,NEW MEXICO,combination long-haul truck,3.906164,CO,diesel,Reference,heavy duty trucks
1,NEW MEXICO,combination short-haul truck,3.323502,CO,diesel,Reference,heavy duty trucks
2,NEW MEXICO,light commercial truck,2.886051,CO,diesel,Reference,light duty trucks
3,NEW MEXICO,light commercial truck,5.687351,CO,gasoline,Reference,light duty trucks
4,NEW MEXICO,passenger car,3.765076,CO,diesel,Reference,light duty autos
...,...,...,...,...,...,...,...
1483,NEW MEXICO,transit bus,0.104000,PM10 (TBW),EREV,AFV,transit buses
1484,NEW MEXICO,transit bus,0.176728,PM2.5,EREV,AFV,transit buses
1485,NEW MEXICO,transit bus,0.013000,PM2.5 (TBW),EREV,AFV,transit buses
1486,NEW MEXICO,transit bus,0.526042,VOC,EREV,AFV,transit buses


In [24]:
#Averages emission factors for short-haul and long-haul trucks, without accounting for their different porportions of total VMT.
all_efs = all_efs.groupby(['evolved_category', 'Pollutant', 'Reference/AFV', 'Fuel Type']).mean().reset_index()

In [25]:
all_efs.to_csv('Evolved/my outputs/{}_allvehicle_efs.csv'.format(state_abbrev))